# Linear Regression

Download the required libraries for this notebook by running the cell below. If you are using conda, uncomment and run the conda command. If you are using PIP, uncomment and use the PIP command.

COME BACK AND MAKE CONDA USE ENVIRONMENT.YML TO LOAD INSTEAD OF REQUIREMENTS.TXT

In [2]:
import sys
#!conda install --yes --prefix {sys.prefix} 
#!{sys.executable} -m pip install -r requirements.txt

In [43]:
import time
import numpy as np
import pandas as pd
from scipy import stats
%matplotlib inline 
import matplotlib as plt

# Ensures that all dataframes are displayed on one line instead of breaking columns across multiple lines
pd.set_option('display.expand_frame_repr', False)

## Import the Data

For this example, the gold medal results of olympic track and field events will be used to show linear regression in action. The men's marathon times from 1896 to 2016 will be used to draw a line of best fit. The original dataset can be found [here](https://www.kaggle.com/datasets/jayrav13/olympic-track-field-results/discussion/272523).

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/wrmthorne/linear-regression/main/results.csv', encoding= 'unicode_escape')
x = np.array(df.iloc[:, 0].values).reshape(-1,1)
y = np.array(df.iloc[:, 1].values).reshape(-1,1)

# Print some information about the dataframe
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2394 entries, 0 to 2393
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Gender       2394 non-null   object 
 1   Event        2394 non-null   object 
 2   Location     2394 non-null   object 
 3   Year         2394 non-null   int64  
 4   Medal        2394 non-null   object 
 5   Name         2164 non-null   object 
 6   Nationality  2394 non-null   object 
 7   Result       2394 non-null   object 
 8   Wind         12 non-null     float64
dtypes: float64(1), int64(1), object(7)
memory usage: 168.5+ KB
None


### Exercise 1

Extract only the men's marathon results from the full list of data and keep only the 'Year' and 'Result' columns. We are only interested in plotting the year against the time achieved for each year.

Hint: The event is called 'Marathon Men'

In [30]:
# Example answer
data = df[['Year', 'Result']].loc[df.Event == 'Marathon Men']

In [ ]:
# Test to see whether your method returned the right results


## Preparing the data

Once we have selected the subset of data we want to use, we need to split it into each separate axis to represent. Since we are only mapping the function of two columns, we can use a function of the form:

$$
y = mx + c
$$

As our independent variable is the year and our dependent variable is the result, we assign $x$ to year and $y$ to time. Because time is in a string format in the dataset, we need to convert it to a numerical form that can be use in arithmetic.

In [44]:
# Show y before transformation
print(y[:5])
# Convert Results column to be in seconds


# Assign x and y and shape them into a column vector
x = np.array(data.Year.values).reshape(-1, 1)
y = np.array(data.Result.values).reshape(-1, 1)
print(y[:5])


[['02:08:44']
 ['2:09:54']
 ['2:10:05']
 ['2h06:32']
 ['2h07:16']]


In [39]:
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

## Extension: The Maths
If you are interested in the maths of linear regression, this section will cover how to implement linear regression manually in numpy. This will also cover the underlying linear algebra involved